In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom
from typing import List, Tuple
import numpy as np
import rdkit
import pandas as pd


def get_3d_coords_from_smiles(smiles:str, add_hydrogen:bool=True, seed=0xf00d):

    m = Chem.MolFromSmiles(smiles)
    # necessary to add hydrogen for consistent conformer generation
    m = Chem.AddHs(m)

    ## 3D conformer generation
    ps = rdDistGeom.ETKDGv3()
    ps.randomSeed = seed
    #ps.coordMap = coordMap = {0:[0,0,0]}
    AllChem.EmbedMolecule(m,ps)


    conf = m.GetConformer()

    ## if we dont want hydrogen, we need to rebuild a molecule without explicit hydrogens
    if not(add_hydrogen):
        m = Chem.MolFromSmiles(smiles)
    
    sub = m.GetSubstructMatch(m)

    ## intiliazing properties dictionary with the 3d coordinates of each atom
    properties = {i:{"atom_name":m.GetAtoms()[s].GetSymbol(), "coordinates":np.array(conf.GetAtomPosition(s)), "bond_adjacency_list":[]} for i,s in enumerate(sub)}

    ## creating adjacency list for bonds 
    for bond in m.GetBonds():
        properties[bond.GetBeginAtomIdx()]["bond_adjacency_list"].append((bond.GetEndAtomIdx(), str(bond.GetBondType())))

    return properties


#
def translate_properties(properties):
    """Translate all 3d coordinates such that the first atom has coordinates (0,0,0)

    Args:
        properties (_type_): _description_

    Returns:
        _type_: _description_
    """
    from copy import deepcopy

    first_atom_coordinates = properties[0]["coordinates"]
    normalized_properties = deepcopy(properties)

    for k in properties:
        normalized_properties[k]["coordinates"] = normalized_properties[k]["coordinates"] - first_atom_coordinates

    return normalized_properties



def print_coordinates(properties):
    """Print 3D coordinates of each atom of a molecule
    """

    for k in properties:
        print(f"Name: {properties[k]['atom_name']}{k} Coords:{properties[k]['coordinates']}")


smiles = 'CC(C)C'

properties = get_3d_coords_from_smiles(smiles=smiles, seed=3, add_hydrogen=True)

print_coordinates(properties=properties)

normalized_properties = translate_properties(properties=properties)

print_coordinates(normalized_properties)


Name: C0 Coords:[ 1.42818766 -0.14813806  0.08477478]
Name: C1 Coords:[-0.03701396 -0.03146544  0.41758567]
Name: C2 Coords:[-0.78503034 -1.08658259 -0.35578434]
Name: C3 Coords:[-0.61566472  1.30330658  0.07065338]
Name: H4 Coords:[2.07037515 0.49551914 0.71512974]
Name: H5 Coords:[ 1.71660132 -1.20892847  0.29642142]
Name: H6 Coords:[ 1.6178553   0.02771058 -1.00275004]
Name: H7 Coords:[-0.12895147 -0.17315105  1.5196666 ]
Name: H8 Coords:[-1.27395018 -1.75945163  0.39039957]
Name: H9 Coords:[-1.57245633 -0.67905774 -1.0048914 ]
Name: H10 Coords:[-0.11544116 -1.70273711 -0.9906863 ]
Name: H11 Coords:[-1.41616048  1.60261002  0.79153569]
Name: H12 Coords:[-1.0294526   1.24483837 -0.96890214]
Name: H13 Coords:[0.14110181 2.11552741 0.03684737]
Name: C0 Coords:[0. 0. 0.]
Name: C1 Coords:[-1.46520162  0.11667262  0.33281089]
Name: C2 Coords:[-2.213218   -0.93844453 -0.44055912]
Name: C3 Coords:[-2.04385239  1.45144464 -0.0141214 ]
Name: H4 Coords:[0.64218748 0.6436572  0.63035495]
Name: 

In [6]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom
from typing import List, Tuple
import numpy as np
import rdkit
import pandas as pd
from pymatgen.core import Structure, Lattice, Molecule, Element


from utils import from_structure_to_molecule
from pymatgen.io.babel import BabelMolAdaptor

from pymatgen.core.lattice import Lattice


%load_ext autoreload
%autoreload 2


smiles = 'CC(C)C'

m = Chem.MolFromSmiles(smiles, sanitize = False)
# necessary to add hydrogen for consistent conformer generation

problems = Chem.DetectChemistryProblems(m)
print(len(problems))



struct = Structure(
    Lattice.cubic(4.2),
    ["Na", "K", "K"],
    [[0, 0, 0], [0.5, 0.5, 0.5], [0,0,0.5]],
    site_properties={"magmom": [-2, 2,2]},
)





0


In [11]:
import json
import gzip
from pymatgen.core.structure import Structure, Molecule

from utils import download_url

#raw_url= "https://ml.materialsproject.org/projects/matbench_mp_is_metal.json.gz"
#raw_dir= "../data/matbench/mp_is_metal"
#download_url(raw_url, raw_dir)


json_filename = "../old_data/matbench/mp_is_metal/raw/matbench_mp_is_metal.json.gz"


with gzip.open(json_filename, 'r') as fin:        # 4. gzip
    json_bytes = fin.read()                      # 3. bytes (i.e. UTF-8)

json_str = json_bytes.decode('utf-8')            # 2. string (i.e. JSON)
data = json.loads(json_str) 


In [25]:
from utils import from_structure_to_molecule
from pymatgen.io.babel import BabelMolAdaptor

from pymatgen.core.lattice import Lattice

for i in range(len(data["data"])):
    struct = Structure.from_dict(data["data"][i][0])


    #then the following conversion : pymatgen.Structure -> pymatgen.Molecule -> pybel_mol -> mol file (to retain 3D information) ->  rdkit molecule
    mol = Molecule(species=struct.species, coords=struct.cart_coords)
    adaptor = BabelMolAdaptor(mol).pybel_mol
    #ideally, we would like to give the correct 3D coordinates to the molecule, so we use .mol file
    mol_file = adaptor.write('mol')

    new_mol = Chem.MolFromMolBlock(mol_file, sanitize=False)
    problems = Chem.DetectChemistryProblems(new_mol)
    len_problems=len(problems)
    
    new_mol.UpdatePropertyCache(strict=False)
    Chem.SanitizeMol(new_mol,Chem.SanitizeFlags.SANITIZE_FINDRADICALS|Chem.SanitizeFlags.SANITIZE_KEKULIZE|Chem.SanitizeFlags.SANITIZE_SETAROMATICITY|Chem.SanitizeFlags.SANITIZE_SETCONJUGATION|Chem.SanitizeFlags.SANITIZE_SETHYBRIDIZATION|Chem.SanitizeFlags.SANITIZE_SYMMRINGS,catchErrors=True)
    problems = Chem.DetectChemistryProblems(new_mol)
    len_problems=len(problems)
    if len_problems > 0:
        print(f"index {i}, problems {problems}")

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond


index 85, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501120>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501430>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501040>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5014a0>)
index 88, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5014a0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5012e0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5010b0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501740>)
index 136, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501430>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501040>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5016d0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501350>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5013c0>, <rdkit.Chem.rdch

*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond


index 158, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5e0b30>,)
index 171, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5013c0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501510>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501430>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501040>)
index 184, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5013c0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501510>)
index 201, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5013c0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501510>)
index 202, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5013c0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501510>)
index 203, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5013c0>, <rdkit.Chem.rdchem._c

*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol

...

*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond


index 10773, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5e0c10>,)
index 10781, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501ba0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5019e0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5017b0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5016d0>)


*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond


index 10788, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5e0dd0>,)
index 10789, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5e0dd0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501f20>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501dd0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5010b0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5013c0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501ac0>)
index 10802, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5e0b30>,)
index 10815, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501f20>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb501dd0>)
index 10818, problems (<rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5016d0>, <rdkit.Chem.rdchem._cppAtomValenceException object at 0x7f78cb5012e0>, <rdkit.Chem.rdchem._cppAtomVale

KeyboardInterrupt: 